In [114]:
import torch 
import argparse
from utils import dotdict
from activation_dataset import setup_token_data
import wandb
import json
from datetime import datetime
from tqdm import tqdm
from einops import rearrange
import matplotlib.pyplot as plt
# from standard_metrics import run_with_model_intervention, perplexity_under_reconstruction, mean_nonzero_activations
# Create 
# # make an argument parser directly below
# parser = argparse.ArgumentParser()
# parser.add_argument("--model_name", type=str, default="EleutherAI/pythia-70m-deduped")
# parser.add_argument("--layer", type=int, default=4)
# parser.add_argument("--setting", type=str, default="residual")
# parser.add_argument("--l1_alpha", type=float, default=3e-3)
# parser.add_argument("--num_epochs", type=int, default=10)
# parser.add_argument("--model_batch_size", type=int, default=4)
# parser.add_argument("--lr", type=float, default=1e-3)
# parser.add_argument("--kl", type=bool, default=False)
# parser.add_argument("--reconstruction", type=bool, default=False)
# parser.add_argument("--dataset_name", type=str, default="NeelNanda/pile-10k")
# parser.add_argument("--device", type=str, default="cuda:4")

# args = parser.parse_args()
cfg = dotdict()
# cfg.model_name="EleutherAI/pythia-70m-deduped", "usvsnsp/pythia-6.9b-sft"
cfg.model_name="EleutherAI/pythia-70m-deduped"
cfg.target_name="EleutherAI/pythia-70m-deduped"
cfg.layers=[4]
cfg.setting="residual"
cfg.tensor_name="gpt_neox.layers.{layer}"
cfg.l1_alpha=5e-3
cfg.sparsity=None
cfg.num_epochs=10
cfg.model_batch_size=4
cfg.lr=1e-3
cfg.kl=False
cfg.reconstruction=False
# cfg.dataset_name="NeelNanda/pile-10k"
cfg.dataset_name="Elriggs/openwebtext-100k"
cfg.device="cuda:0"
cfg.ratio = 4
# cfg.device="cpu"

In [115]:
tensor_names = [cfg.tensor_name.format(layer=layer) for layer in cfg.layers]

In [116]:
# Load in the model
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(cfg.model_name)
model = model.to(cfg.device)
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)

In [117]:
# Download the dataset
# TODO iteratively grab dataset?
cfg.max_length = 256
cfg.model_batch_size = 16
token_loader = setup_token_data(cfg, tokenizer, model)
num_tokens = cfg.max_length*cfg.model_batch_size*len(token_loader)
print(f"Number of tokens: {num_tokens}")

Found cached dataset parquet (/root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/Elriggs___parquet/Elriggs--openwebtext-100k-79076ecafee8a6d5/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-8401ec7d4dbd84d2_*_of_00008.arrow


Number of tokens: 112750592


In [118]:
# Run 1 datapoint on model to get the activation size
from baukit import Trace

text = "1"
tokens = tokenizer(text, return_tensors="pt").input_ids.to(cfg.device)
# Your activation name will be different. In the next cells, we will show you how to find it.
with torch.no_grad():
    with Trace(model, tensor_names[0]) as ret:
        _ = model(tokens)
        representation = ret.output
        # check if instance tuple
        if(isinstance(representation, tuple)):
            representation = representation[0]
        activation_size = representation.shape[-1]
print(f"Activation size: {activation_size}")

Activation size: 512


In [119]:
# Initialize New autoencoder
from autoencoders.learned_dict import TiedSAE, UntiedSAE, AnthropicSAE
from torch import nn
params = dict()
n_dict_components = activation_size*cfg.ratio
params["encoder"] = torch.empty((n_dict_components, activation_size), device=cfg.device)
nn.init.xavier_uniform_(params["encoder"])

params["decoder"] = torch.empty((n_dict_components, activation_size), device=cfg.device)
nn.init.xavier_uniform_(params["decoder"])

params["encoder_bias"] = torch.empty((n_dict_components,), device=cfg.device)
nn.init.zeros_(params["encoder_bias"])

params["shift_bias"] = torch.empty((activation_size,), device=cfg.device)
nn.init.zeros_(params["shift_bias"])

autoencoder = TiedSAE(  # Note: this has no decoder bias
    # n_feats = n_dict_components, 
    # activation_size=activation_size,
    encoder=params["encoder"],
    encoder_bias=params["encoder_bias"],
    # decoder=params["decoder"],
    # shift_bias=params["shift_bias"],
)
autoencoder.to_device(cfg.device)
autoencoder.encoder.requires_grad = True
autoencoder.encoder_bias.requires_grad = True
# autoencoder.decoder.requires_grad = True
# autoencoder.shift_bias.requires_grad = True
optimizer = torch.optim.Adam(
    [
        autoencoder.encoder, 
        autoencoder.encoder_bias,
        # autoencoder.decoder,
        # autoencoder.shift_bias,
    ], lr=cfg.lr)

In [120]:
# Set target sparsity to 10% of activation_size if not set
if cfg.sparsity is None:
    cfg.sparsity = int(activation_size*0.1)
    print(f"Target sparsity: {cfg.sparsity}")

target_lower_sparsity = cfg.sparsity - 5.0
target_upper_sparsity = cfg.sparsity + 5.0
adjustment_factor = 0.1  # You can set this to whatever you like

Target sparsity: 51


In [121]:
original_bias = autoencoder.encoder_bias.clone().detach()
# Wandb setup
secrets = json.load(open("secrets.json"))
wandb.login(key=secrets["wandb_key"])
start_time = datetime.now().strftime("%Y%m%d-%H%M%S")
wandb_run_name = f"{cfg.model_name}_{start_time[4:]}_{cfg.sparsity}"  # trim year
print(f"wandb_run_name: {wandb_run_name}")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb_run_name: EleutherAI/pythia-70m-deduped_1008-222616_51


In [122]:
wandb.init(project="sparse coding", config=dict(cfg), name=wandb_run_name)

Dead Features,▁▁▁▁▁▁▁▁▁▁
L1 Loss,█▁▁▁▁▁▁▁▁▁
Reconstruction Loss,█▁▁▁▁▁▁▁▁▁
Self Similarity,█▁▃▄▅▅▆▇▇▇
Sparsity,█▂▂▁▁▁▁▁▁▁
Tokens,▁▂▃▃▄▅▆▆▇█
Total Loss,█▁▁▁▁▁▁▁▁▁
l1_alpha,▁▁▁▁▁▁▁▁▁▁
Dead Features,0
L1 Loss,0.05667
Reconstruction Loss,0.04423


In [123]:
dead_features = torch.zeros(autoencoder.encoder.shape[0])
max_num_tokens = 100000000
saved_activations = []
# Freeze model parameters 
model.eval()
model.requires_grad_(False)
model.to(cfg.device)
last_encoder = autoencoder.encoder.clone().detach()
for i, batch in enumerate(tqdm(token_loader)):
    tokens = batch["input_ids"].to(cfg.device)
    with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
        with Trace(model, tensor_names[0]) as ret:
            _ = model(tokens)
            representation = ret.output
            if(isinstance(representation, tuple)):
                representation = representation[0]
    layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")
    saved_activations.append(layer_activations.clone().cpu().detach())

    c = autoencoder.encode(layer_activations)
    x_hat = autoencoder.decode(c)
    
    reconstruction_loss = (x_hat - layer_activations).pow(2).mean()
    l1_loss = torch.norm(c, 1, dim=-1).mean()
    total_loss = reconstruction_loss + cfg.l1_alpha*l1_loss

    dead_features += c.sum(dim=0).cpu()
    if (i % 500 == 0): # Check here so first check is model w/o change
        # self_similarity = torch.cosine_similarity(c, last_encoder, dim=-1).mean().cpu().item()
        # Above is wrong, should be similarity between encoder and last encoder
        self_similarity = torch.cosine_similarity(autoencoder.encoder, last_encoder, dim=-1).mean().cpu().item()
        last_encoder = autoencoder.encoder.clone().detach()
        num_tokens_so_far = i*cfg.max_length*cfg.model_batch_size
        with torch.no_grad():
            sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
            # Count number of dead_features are zero
            num_dead_features = (dead_features == 0).sum().item()
        print(f"Sparsity: {sparsity:.1f} | Dead Features: {num_dead_features} | Total Loss: {total_loss:.2f} | Reconstruction Loss: {reconstruction_loss:.2f} | L1 Loss: {cfg.l1_alpha*l1_loss:.2f} | l1_alpha: {cfg.l1_alpha:.2e} | Tokens: {num_tokens_so_far} | Self Similarity: {self_similarity:.2f}")
        wandb.log({
            'Sparsity': sparsity,
            'Dead Features': num_dead_features,
            'Total Loss': total_loss.item(),
            'Reconstruction Loss': reconstruction_loss.item(),
            'L1 Loss': (cfg.l1_alpha*l1_loss).item(),
            'l1_alpha': cfg.l1_alpha,
            'Tokens': num_tokens_so_far,
            'Self Similarity': self_similarity
        })
        
        # # RESAMPLING
        # if num_dead_features > 0:
        #     # hyperparams:
        #     max_resample_tokens = 1000
        #     # compute loss of model on random subset of inputs
        #     i=10
        #     resample_loader = setup_token_data(cfg, tokenizer, model, seed=i)
        #     num_resample_data = 0

        #     resample_activations = torch.empty(0, activation_size)
        #     resample_losses = torch.empty(0)

        #     for resample_batch in resample_loader:
        #         resample_tokens = resample_batch["input_ids"].to(cfg.device)
        #         with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
        #             with Trace(model, tensor_names[0]) as ret:
        #                 _ = model(resample_tokens)
        #                 representation = ret.output
        #                 if(isinstance(representation, tuple)):
        #                     representation = representation[0]
        #         layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")
        #         resample_activations = torch.cat((resample_activations, layer_activations.detach().cpu()), dim=0)

        #         c = autoencoder.encode(layer_activations)
        #         x_hat = autoencoder.decode(c)
                
        #         reconstruction_loss = (x_hat - layer_activations).pow(2).mean(dim=-1)
        #         l1_loss = torch.norm(c, 1, dim=-1)
        #         total_loss = reconstruction_loss + cfg.l1_alpha*l1_loss
                
        #         resample_losses = torch.cat((resample_losses, total_loss.detach().cpu()), dim=0)
                
        #         num_resample_data +=layer_activations.shape[0]
        #         if num_resample_data > max_resample_tokens:
        #             break

                
        #     # sample num_dead_features vectors of input activations
        #     probabilities = resample_losses**2
        #     sampled_indices = torch.multinomial(probabilities, num_dead_features)
        #     new_vectors = resample_activations[sampled_indices]

        #     # calculate average encoder norm of alive neurons
        #     alive_neurons = list((dead_features!=0))
        #     modified_columns = dead_features==0
        #     avg_norm = autoencoder.encoder.data[alive_neurons].norm(dim=-1).mean()

        #     # replace dictionary and encoder weights with vectors
        #     new_vectors = new_vectors / new_vectors.norm(dim=1, keepdim=True)
            
        #     params_to_modify = [autoencoder.encoder, autoencoder.encoder_bias]

        #     current_weights = autoencoder.encoder.data
        #     current_weights[modified_columns] = new_vectors * avg_norm * 0.2
        #     autoencoder.encoder.data = current_weights

        #     current_weights = autoencoder.encoder_bias.data
        #     current_weights[modified_columns] = 0
        #     autoencoder.encoder_bias.data = current_weights
            
        #     if hasattr(autoencoder, 'decoder'):
        #         current_weights = autoencoder.decoder.data
        #         current_weights[modified_columns] = new_vectors
        #         autoencoder.decoder.data = current_weights
        #         params_to_modify += [autoencoder.decoder]

        #     for param_group in optimizer.param_groups:
        #         for param in param_group['params']:
        #             if param in params_to_modify:
        #                 # Extract the corresponding rows from m and v
        #                 m = optimizer.state[param]['exp_avg']
        #                 v = optimizer.state[param]['exp_avg_sq']
                        
        #                 # Update the m and v values for the modified columns
        #                 m[modified_columns] = 0  # Reset moving average for modified columns
        #                 v[modified_columns] = 0  # Reset squared moving average for modified columns
        
        dead_features = torch.zeros(autoencoder.encoder.shape[0])
        
        if(num_tokens_so_far > max_num_tokens):
            print(f"Reached max number of tokens: {max_num_tokens}")
            break

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # # Running sparsity check
    # if(num_tokens_so_far > 500000):
    #     if(i % 200 == 0):
    #         with torch.no_grad():
    #             sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
    #         if sparsity > target_upper_sparsity:
    #             cfg.l1_alpha *= (1 + adjustment_factor)
    #         elif sparsity < target_lower_sparsity:
    #             cfg.l1_alpha *= (1 - adjustment_factor)
    #         # print(f"Sparsity: {sparsity:.1f} | l1_alpha: {cfg.l1_alpha:.2e}")

  0%|          | 1/27527 [00:00<54:17,  8.45it/s]

  0%|          | 3/27527 [00:00<39:04, 11.74it/s]

Sparsity: 1030.8 | Dead Features: 0 | Total Loss: 2.39 | Reconstruction Loss: 0.51 | L1 Loss: 1.88 | l1_alpha: 5.00e-03 | Tokens: 0 | Self Similarity: 1.00


  2%|▏         | 503/27527 [00:37<33:46, 13.34it/s]

Sparsity: 11.7 | Dead Features: 0 | Total Loss: 0.30 | Reconstruction Loss: 0.22 | L1 Loss: 0.08 | l1_alpha: 5.00e-03 | Tokens: 2048000 | Self Similarity: 0.90


...

Sparsity: 18.5 | Dead Features: 0 | Total Loss: 0.23 | Reconstruction Loss: 0.12 | L1 Loss: 0.10 | l1_alpha: 5.00e-03 | Tokens: 59392000 | Self Similarity: 0.97


 55%|█████▍    | 15003/27527 [19:22<16:15, 12.84it/s]

Sparsity: 18.8 | Dead Features: 0 | Total Loss: 0.23 | Reconstruction Loss: 0.12 | L1 Loss: 0.10 | l1_alpha: 5.00e-03 | Tokens: 61440000 | Self Similarity: 0.97


 55%|█████▍    | 15105/27527 [19:30<16:02, 12.91it/s]


KeyboardInterrupt: 

In [20]:
model_save_name = cfg.model_name.split("/")[-1]
save_name = f"{model_save_name}_sp{cfg.sparsity}_r{cfg.ratio}_{tensor_names[0]}"  # trim year

# Make directory traiend_models if it doesn't exist
import os
if not os.path.exists("trained_models"):
    os.makedirs("trained_models")
# Save model
torch.save(autoencoder, f"trained_models/{save_name}.pt")

# if not os.path.exists("activations"):
#     os.makedirs("activations")
# # Save model
# torch.save(saved_activations[:-1], f"activations/{save_name}.pt")

RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 95545616512 vs 95545616408

In [103]:
wandb.finish()

Dead Features,▁▁▁▁▁▁▁▁▁▁
L1 Loss,█▁▁▁▁▁▁▁▁▁
Reconstruction Loss,█▁▁▁▁▁▁▁▁▁
Self Similarity,█▁▃▄▅▅▆▇▇▇
Sparsity,█▂▂▁▁▁▁▁▁▁
Tokens,▁▂▃▃▄▅▆▆▇█
Total Loss,█▁▁▁▁▁▁▁▁▁
l1_alpha,▁▁▁▁▁▁▁▁▁▁
Dead Features,0
L1 Loss,0.05667
Reconstruction Loss,0.04423


In [ ]:
# cfg.model_name="EleutherAI/pythia-70m-deduped"
cfg.model_name="usvsnsp/pythia-6.9b-sft"
cfg.target_name="usvsnsp/pythia-6.9b-ppo"
cfg.layers=[4]
cfg.setting="residual"
cfg.tensor_name="gpt_neox.layers.{layer}"
cfg.l1_alpha=1e-3
cfg.sparsity=None
cfg.num_epochs=10
cfg.model_batch_size=4
cfg.lr=1e-3
cfg.kl=False
cfg.reconstruction=False
# cfg.dataset_name="NeelNanda/pile-10k"
cfg.dataset_name="Elriggs/openwebtext-100k"
cfg.device="cuda:0"
cfg.ratio = 4
# cfg.device="cpu"

model = model.cpu()
target_model = AutoModelForCausalLM.from_pretrained(cfg.target_name)
target_model = target_model.to(cfg.device)

In [ ]:
# Initialize New autoencoder
from autoencoders.learned_dict import TiedSAE, UntiedSAE
from torch import nn

# params["decoder"] = torch.empty((n_dict_components, activation_size), device=cfg.device)
# nn.init.xavier_uniform_(params["decoder"])

transfer_autoencoder = UntiedSAE(
    # n_feats = n_dict_components, 
    # activation_size=activation_size,
    encoder=autoencoder.encoder,
    encoder_bias=autoencoder.encoder_bias,
    decoder=autoencoder.encoder.clone(),
)
transfer_autoencoder.to_device(cfg.device)

# Set gradient to true for decoder only- only training decoder on transfer
transfer_autoencoder.encoder.requires_grad = False
transfer_autoencoder.encoder_bias.requires_grad = False
transfer_autoencoder.decoder.requires_grad = True
optimizer = torch.optim.Adam([transfer_autoencoder.decoder], lr=1e-3)


In [ ]:
# Wandb setup
secrets = json.load(open("secrets.json"))
wandb.login(key=secrets["wandb_key"])
start_time = datetime.now().strftime("%Y%m%d-%H%M%S")
wandb_run_name = f"{cfg.target_name}_transfer_{start_time[4:]}_{cfg.sparsity}"  # trim year
print(f"wandb_run_name: {wandb_run_name}")

In [ ]:
# Training transfer autoencoder
dead_features = torch.zeros(transfer_autoencoder.encoder.shape[0])
max_num_tokens = 100000000
# Freeze model parameters 
model = model.cpu()
target_model = target_model.to(cfg.device)
target_model.eval()
target_model.requires_grad_(False)
last_encoder = transfer_autoencoder.encoder.clone().detach()
for i, (batch, base_activations) in enumerate(tqdm(zip(token_loader, saved_activations), total=min(len(token_loader),len(saved_activations)))):
    tokens = batch["input_ids"].to(cfg.device)
    with torch.no_grad(): # As long as not doing KL divergence, don't need gradients for model
        with Trace(target_model, tensor_names[0]) as ret:
            _ = target_model(tokens)
            representation = ret.output
            if(isinstance(representation, tuple)):
                representation = representation[0]
    # layer_activations will be the output of the autoencoder
    layer_activations = rearrange(representation, "b seq d_model -> (b seq) d_model")

    c = transfer_autoencoder.encode(base_activations.to(cfg.device))
    x_hat = transfer_autoencoder.decode(c)
    
    reconstruction_loss = (x_hat - layer_activations).pow(2).mean()
    l1_loss = torch.norm(c, 1, dim=-1).mean()
    total_loss = reconstruction_loss + cfg.l1_alpha*l1_loss

    dead_features += c.sum(dim=0).cpu()
    if (i % 500 == 0): # Check here so first check is model w/o change
        # self_similarity = torch.cosine_similarity(c, last_encoder, dim=-1).mean().cpu().item()
        # Above is wrong, should be similarity between encoder and last encoder
        self_similarity = torch.cosine_similarity(transfer_autoencoder.encoder, last_encoder, dim=-1).mean().cpu().item()
        last_encoder = transfer_autoencoder.encoder.clone().detach()
        num_tokens_so_far = i*cfg.max_length*cfg.model_batch_size
        with torch.no_grad():
            sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
            # Count number of dead_features are zero
            num_dead_features = (dead_features == 0).sum().item()
        print(f"Sparsity: {sparsity:.1f} | Dead Features: {num_dead_features} | Total Loss: {total_loss:.2f} | Reconstruction Loss: {reconstruction_loss:.2f} | L1 Loss: {cfg.l1_alpha*l1_loss:.2f} | l1_alpha: {cfg.l1_alpha:.2e} | Tokens: {num_tokens_so_far} | Self Similarity: {self_similarity:.2f}")
        wandb.log({
            'Sparsity': sparsity,
            'Dead Features': num_dead_features,
            'Total Loss': total_loss.item(),
            'Reconstruction Loss': reconstruction_loss.item(),
            'L1 Loss': (cfg.l1_alpha*l1_loss).item(),
            'l1_alpha': cfg.l1_alpha,
            'Tokens': num_tokens_so_far,
            'Self Similarity': self_similarity
        })
        dead_features = torch.zeros(transfer_autoencoder.encoder.shape[0])
        
        if(num_tokens_so_far > max_num_tokens):
            print(f"Reached max number of tokens: {max_num_tokens}")
            break

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # Running sparsity check
    if(num_tokens_so_far > 500000):
        if(i % 200 == 0):
            with torch.no_grad():
                sparsity = (c != 0).float().mean(dim=0).sum().cpu().item()
            if sparsity > target_upper_sparsity:
                cfg.l1_alpha *= (1 + adjustment_factor)
            elif sparsity < target_lower_sparsity:
                cfg.l1_alpha *= (1 - adjustment_factor)
            # print(f"Sparsity: {sparsity:.1f} | l1_alpha: {cfg.l1_alpha:.2e}")

In [ ]:
model_save_name = cfg.target_name.split("/")[-1]
save_name = f"{model_save_name}_transfer_sp{cfg.sparsity}_r{cfg.ratio}_{tensor_names[0]}"  # trim year

# Make directory traiend_models if it doesn't exist
import os
if not os.path.exists("trained_models"):
    os.makedirs("trained_models")
# Save model
torch.save(transfer_autoencoder, f"trained_models/{save_name}.pt")

In [ ]:
wandb.finish()

In [ ]:
cfg

{'model_name': 'EleutherAI/pythia-70m-deduped',
 'layers': [4],
 'setting': 'residual',
 'tensor_name': 'gpt_neox.layers.{layer}',
 'l1_alpha': 0.0020591228579666505,
 'sparsity': 51,
 'num_epochs': 10,
 'model_batch_size': 4,
 'lr': 0.001,
 'kl': False,
 'reconstruction': False,
 'dataset_name': 'NeelNanda/pile-10k',
 'device': 'cuda:0',
 'ratio': 4,
 'max_length': 256}